In [1]:
# Code is taken from : https://pennylane.ai/qml/demos/tutorial_backprop.html

In [2]:
import numpy as np
import pennylane as qml

In [3]:
np.random.seed(42)

dev=qml.device("default.qubit",wires=3)

@qml.qnode(dev, diff_method="parameter-shift")
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RZ(params[2], wires=2)

    qml.broadcast(qml.CNOT, wires=[0, 1, 2], pattern="ring") # entangelment leyer

    qml.RX(params[3], wires=0)
    qml.RY(params[4], wires=1)
    qml.RZ(params[5], wires=2)

    qml.broadcast(qml.CNOT, wires=[0, 1, 2], pattern="ring")
    return qml.expval(qml.PauliY(0) @ qml.PauliZ(2))

In [4]:
params = np.random.random([6])

print("Parameters:", params)
print("Expectation value:", circuit(params))

print(circuit.draw())

Parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]
Expectation value: -0.11971365706871565
 0: ──RX(0.375)──╭C─────────────────╭X──RX(0.599)──╭C──────╭X──╭┤ ⟨Y ⊗ Z⟩ 
 1: ──RY(0.951)──╰X──╭C──RY(0.156)──│──────────────╰X──╭C──│───│┤         
 2: ──RZ(0.732)──────╰X─────────────╰C──RZ(0.156)──────╰X──╰C──╰┤ ⟨Y ⊗ Z⟩ 



In [5]:
# shift parameters 
def parameter_shift_term(qnode, params, i):
    shifted = params.copy()
    shifted[i] += np.pi/2
    forward = qnode(shifted)  # forward evaluation

    shifted[i] -= np.pi
    backward = qnode(shifted) # backward evaluation

    return 0.5 * (forward - backward)

# gradient with respect to the first parameter
print(parameter_shift_term(circuit, params, 0))

-0.06518877224958125


In [6]:
def parameter_shift(qnode, params):
    gradients = np.zeros([len(params)])

    for i in range(len(params)):
        gradients[i] = parameter_shift_term(qnode, params, i)

    return gradients

print(parameter_shift(circuit, params))

[-6.51887722e-02 -2.72891905e-02  0.00000000e+00 -9.33934621e-02
 -7.61067572e-01  4.16333634e-17]


In [7]:
grad_function = qml.grad(circuit)
print(grad_function(params)[0])

[-6.51887722e-02 -2.72891905e-02  0.00000000e+00 -9.33934621e-02
 -7.61067572e-01  4.16333634e-17]


In [8]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev, diff_method="parameter-shift", mutable=False)
def circuit(params):
    qml.templates.StronglyEntanglingLayers(params, wires=[0, 1, 2, 3])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3))

In [9]:
# initialize circuit parameters
params = qml.init.strong_ent_layers_normal(n_wires=4, n_layers=15)
print(params.size)
print(circuit(params))

180
0.8947771876917632


In [10]:
import timeit

repeat = 3
number = 10
times = timeit.repeat("circuit(params)", globals=globals(), number=number, repeat=repeat)
forward_time = min(times) / number

print(f"Forward pass (best of {repeat}): {forward_time} sec per loop")

Forward pass (best of 3): 0.009449109999999905 sec per loop


In [11]:
grad_fn = qml.grad(circuit)

times = timeit.repeat("grad_fn(params)", globals=globals(), number=number, repeat=repeat)
backward_time = min(times) / number

print(f"Gradient computation (best of {repeat}): {backward_time} sec per loop")

Gradient computation (best of 3): 3.7039894900000006 sec per loop


In [13]:
!pip install tensorflow

  Using cached tensorflow-2.3.1-cp37-cp37m-win_amd64.whl (342.5 MB)
  Using cached protobuf-3.13.0-cp37-cp37m-win_amd64.whl (1.0 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached absl_py-0.10.0-py3-none-any.whl (127 kB)
Processing c:\users\tomut\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2\termcolor-1.1.0-py3-none-any.whl
  Using cached grpcio-1.33.1-cp37-cp37m-win_amd64.whl (2.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_auth-1.22.1-py2.py3-none-any.whl (114 kB)
  Using cached google_auth_oauthlib-0.4.1-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_pl

In [14]:
#Backpropagation
import tensorflow as tf

dev = qml.device("default.qubit.tf", wires=4)

In [15]:
def circuit(params):
    qml.templates.StronglyEntanglingLayers(params, wires=[0, 1, 2, 3])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3))

# initialize circuit parameters
params = qml.init.strong_ent_layers_normal(n_wires=4, n_layers=15)
params = tf.Variable(params)
print(circuit(params))

TypeError: Rot: Real scalar parameter expected, got <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [ ]:
import timeit

repeat = 3
number = 10
times = timeit.repeat("circuit(params)", globals=globals(), number=number, repeat=repeat)
forward_time = min(times) / number
print(f"Forward pass (best of {repeat}): {forward_time} sec per loop")

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    res = circuit(params)

times = timeit.repeat("tape.gradient(res, params)", globals=globals(), number=number, repeat=repeat)
backward_time = min(times) / number
print(f"Backward pass (best of {repeat}): {backward_time} sec per loop")